<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9C%A0%EC%98%88%EC%8A%AC_keybert_%EB%8B%B5%EB%B3%80_%EC%83%9D%EC%84%B1_AI_(%EC%A7%84%ED%96%89%EC%A4%91).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 특성 추출 사전학습 모델

- skt/kobert-base-v1 (131k download)
- BM-K/KoSimCSE-roberta (87k download)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# DATA_PATH = "/content/drive/MyDrive/생성 AI 모델링/data/"
DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device




In [ ]:
yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi_all.csv")


In [ ]:
google_food = pd.read_csv(f"{DATA_PATH}playstore_food_reply_0904.csv")

In [ ]:
# yogiyo_added =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905.csv")


In [ ]:
train= yogiyo[['고객리뷰','사장댓글']]

rename_cols = ['review', 'reply']
train.columns = rename_cols
train.head()

In [ ]:
review_list = []
for review in train['review']:
    if isinstance(review, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

In [ ]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

- 긍부정지수 추가

In [ ]:
train_posneg = pd.read_csv(f"{DATA_PATH}train_posneg.csv")

In [ ]:
cols = ['review','Unnamed: 0']
train_posneg = train_posneg.drop(columns=cols)
train_posneg

In [ ]:
train = pd.concat([train,train_posneg],axis=1,ignore_index=True)
train.columns = ['review','reply','sentiment','score']

- 임시 3000개 추출

In [ ]:
mask = train['sentiment'] =='부정'
train_neg = train[mask] #893개
train_neg

In [ ]:
mask = train['sentiment'] =='긍정'
train_pos = train[mask] #17793개
train_pos

In [ ]:
shuffled_neg = train_neg.sample(frac=1)

In [ ]:
shuffled_pos = train_pos.sample(frac=1)[:3000-shuffled_neg.shape[0]]

In [ ]:
shuffled_train = pd.concat([shuffled_neg, shuffled_pos],axis=0)
shuffled_train = shuffled_train.sample(frac=1)
shuffled_train.shape

In [ ]:
shuffled_train

In [ ]:
shuffled_train.isna().sum()

In [ ]:
shuffled_train.shape

# Keybert (skt kobert)




In [ ]:
!pip install keybert

In [ ]:
from keybert import KeyBERT
# from transformers import XLNetTokenizer, BertTokenizer, BertModel
from tqdm import tqdm  # tqdm 추가
from transformers import AutoModel, AutoTokenizer


# keybert에 사용할 Kobert 모델 로드
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
model = AutoModel.from_pretrained("skt/kobert-base-v1")

# KeyBERT 모델 초기화
keybert_model = KeyBERT(model=model)

# 리뷰 데이터 (train['review']로 가정)
reviews = shuffled_train['review'] # 셔플로 섞은 3000개에 대해서 키워드 추출함
keywords_list = []

# tqdm을 사용하여 진행 상황 표시
for text in tqdm(reviews):
    keywords = keybert_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
    keywords = [keyword[0] for keyword in keywords]
    keywords_list.append(keywords)

# # 결과 출력
# for idx, keywords in enumerate(keywords_list):
#     print(f"Review {idx + 1} Keywords:", keywords)

result_df = pd.DataFrame({'Keywords': keywords_list})
result_df


In [ ]:
result_df.to_csv(f"{DATA_PATH}train_3000_keyword_kobert.csv",index=False,encoding='utf-8')

In [ ]:
x = pd.read_csv(f"{DATA_PATH}train_3000_keyword_kobert.csv")

In [ ]:
tmp = result_df.reset_index()[['Keywords']]
tmp

In [ ]:
tmp.isna().sum()

In [ ]:
tmp['Keywords'] = tmp['Keywords'].apply(lambda x: ', '.join(x))

In [ ]:
# df = pd.read_csv(f"{DATA_PATH}train_3000_keyword_kobert.csv")

In [ ]:
tmp = pd.DataFrame(tmp)
tmp

In [ ]:
train = shuffled_train.reset_index()[['review','reply','sentiment','score']]
train

In [ ]:
tmp = pd.concat([train, tmp],axis=1,ignore_index=True)

In [ ]:
tmp

- Keyword shuffled train 3000

In [ ]:
train.isna().sum()

In [ ]:
train_keyword = tmp[[4,1,2,3]]
train_keyword.columns = ['review','reply','sentiment','score']
train_keyword.isna().sum()


In [ ]:
train_keyword.to_csv(f'{DATA_PATH}keyword_shuffled_train_3000_0904.csv',index=False,encoding='utf-8')

- Sentence shuffled train 3000

In [ ]:
train_original = tmp[[0,1,2,3]]
train_original
train_keyword.isna().sum()

In [ ]:
train_original.to_csv(f'{DATA_PATH}original_shuffled_train_3000_0904.csv',index=False,encoding='utf-8')

# ------학습, 검증, 추론

# Keyword shuffled train 3000

In [ ]:
x = pd.read_csv(f'{DATA_PATH}keyword_shuffled_train_3000_0904.csv',encoding='utf-8')

In [ ]:
train_keyword.info()

In [ ]:
x.info()

In [ ]:
x.isna().sum() # 저장시에는 review에 결측치 없었는데, 불러오니까 결측치 생김. 인코딩 문제는 아님.형식문제? 확인 필요

## 전처리

In [ ]:
train = train_keyword.copy()
train.isnull().sum()

#### 고객 id 처리

In [ ]:
train = pd.DataFrame(train)
# train['reply'] = train['reply'].str.replace(r'\w+\s*님', '고객님', regex=True)
train['reply'] = train['reply'].str.replace(r'\w+\s*님|\w+\s*고객님', '고객님', regex=True)

#### 가게명 처리

In [ ]:
ygy_store_lst = ["1988응답하라추억의옛날도시락-방이점", "워커스커피 로스터스", "떡깨비-가락점", "당치땡-헬리오시티점", "우시장국밥-석촌점",  "고돼지-송파점",  "마라의신마라탕&마라샹궈",  "춘리마라탕-송파점",  "손수경의육회담은연어-가락점",  "무국적식탁-송파점",  "청년치킨-삼전점",  "송파만다린",  "배달의쌀국수-송파점",  "닭강점-헬리오시티점",  "토핑폭탄김치찜&김치찌개-송파점",  "조형훈족발보쌈",  "여왕의브런치",  "모어댄버터",  "플러스82-송파점",  "더티베지",  "잭아저씨족발보쌈-본점",  "육회&연어-미아본점",  "갈비민족-본점",  "구구족-성신여대역점",  "피자보이시나-대학로점",  "왕빈자삼파전-미아본점",  "행복한찜닭-성북점",  "스시사소우",  "저팔계&중국&마라탕&덮밥&요리&전문점",  "명작파스타-성북점",  "올데이파스타-동대문구점",  "대한냉면-성북점",  "인생제육",  "고기듬뿍미트박스-성북점",  "피자스웨그-성북점",  "청춘식당묵은지김치찜",  "삽교원조두리곱창",  "뜸들이다-북서울꿈의숲점",  "더팔당매운오징어&닭발 김치찜",  "우리집반찬도시락-혜화막내딸지점",  "수피자",  "마라쿡",  "김포불닭발",  "헬로팬닭갈비&마약볶음밥",  "대찌 대파생고기김치찌개&정육왕국물두루치기",  "구래상회",  "승도리네곱도리탕",  "자성당",  "수락",  "나인곱창",  "어시장",  "돈까스회관",  "호랑이초밥",  "땡초곱창막창",  "피자이탈리",  "텐텐마라탕",  "과일에반하다",  "삼다수",  "병천토속순대",  "최고남제육명가",  "짱닭치킨",  "걸작떡볶이",  "직화삼겹직구삼",  "짜글이가돼었소",  "부대찌개자신있는집",  "토핑폭탄김치찜&김치찌개-부천점",  "1989마라탕-부천점",  "미친피자-부평점",  "천년닭강정-부천부평점",  "수상한삼겹살-부천점",  "수제죽전문점-본가진죽-본점",  "오늘은 분식",  "이끌림마라탕",  "탐나는피자-부평점",  "고기혁명-부천점",  "라화방마라탕-신중동점",  "전금례닭볶음탕-부천점",  "큰아들백순대",  "공주닭발-부천시청점",  "신자매김치찜&김치찌개",  "짱닭치킨-도촌점",  "이태리면가게-야탑점",  "폴트버거-판교점",  "랜돌프뉴욕페페로니-야탑점",  "홍싸롱-수제돈까스&파스타",  "쇼부덮밥",  "카산도-정자본점",  "부성초밥-미금본점",  "닭장수섭삼계탕&전기구이통닭",  "깨돌이김밥-미금점",  "울진죽변항",  "GTS버거-분당정자점",  "코브라독스-분당미금역점",  "마라하오-죽전점",  "땅끝수산",  "초밥대통령-용인수지점",  "올데이케밥&샐러드",  "야키토리 카마쿠라",  "비오키친-서현본점",  "부산오빠1인국밥-분당점",  "알촌",  "밈피자",  "고씨네카레",  "홍스족발",  "뜸들이다",  "감성커피",  "짚신스시",  "떡형",  "39도시락",  "피제이피자",  "황궁쟁반짜장",  "OK반점",  "모모타코야키&카페",  "꼬알라파이",  "진지덮밥",  "피자프라텔로",  "한식세끼1인김치찜&김치찌개",  "윤희횟집",  "시민보쌈족발&감자탕",  "카페봄봄",  "킹프레소빅와플",  "최가네한쌈",  "창타이누들",  "보돌미역",  "홍대개미"  ]


In [ ]:
store_name = []

for store in ygy_store_lst:
    pat = re.split(r'[- &]', store)
    store_name.append(pat[0])

len(store_name)

In [ ]:
store_name[:10]

In [ ]:
# train['store'] = ""  # full store name을 칼럼에 담기
# for name in store_name:
#     train[name] = train['reply'].apply(lambda x: name if name in x else '') # 우선 각 가게명 컬럼 생성(그렇게해야 덮어쓰기가 안됨)
#     train['store'] = train.apply(lambda row: row['store'] + ',' + row[name] if row[name] else row['store'], axis=1) # 하나로 합치기(맨 앞에 콤마가 생김)

# train['store'] = train['store'].apply(lambda x: x[1:] if x.startswith(',') else x) # 맨 앞에 콤마 제거
# train = train.drop(columns=store_name) # 각 가게명 컬럼 삭제

# train.head()

- 1차 : store 이름이 풀로 들어간 경우 <store>로 대체

In [ ]:
def replace_store_names1(text):
    for name1 in store_name:
        if name1 in text:
            # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<store>", text)
    return text

In [ ]:
replace_store_names1("1988응답하라추억의옛날도시락에서 먹은 음식은 정말 맛있었습니다.")

In [ ]:
replace_store_names1("우시장에서 먹은 음식은 정말 맛있었습니다.") # 부분이름은 안바뀜

- 2차 : store 이름이 부분적으로 들어간 경우 store로 대체

In [ ]:
#예시

name1 = '손수경의육회담은연어'
for i in range(len(name1)-1, 1, -1):
    part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
    print(part)

In [ ]:
# # 뒤에서 앞으로
# for i in range(2, len(name1) + 1):
#     part = name1[-i:]
#     print(part)

In [ ]:
def replace_store_names2(text):
    for name1 in tqdm(store_name, desc="Processing"):
        for i in range(len(name1)-1, 1, -1):
            part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분. 글자 긴경우 부터 적용됨
            if part in text:
                # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
                pattern = re.compile(re.escape(part), re.IGNORECASE)
                text = pattern.sub("<store>", text)
    return text

In [ ]:
replace_store_names2("텐텐에서 먹은 음식은 정말 맛있었습니다.") # 텐텐마라탕

In [ ]:
replace_store_names2("손수경에서 먹은 음식은 정말 맛있었습니다.") # 손수경의육회담은연어

- 3차 : 일반명사/형용사 등은 제외하고 대체해야 함

In [ ]:
!pip install kiwipiepy

In [ ]:
# # short_name 에서 가게명 이외 일상용어로 사용되는 키워드는 제거
# drop_name = ['고기','OK','감성','걸작','공주','과일','김포','대한','땡초','명작','미친','배달','병천','부대','부산','비오','삽교',
#              '손수','송파','쇼부','수상','수제','시민','오늘', '오늘은', '우리', '울진', '이끌', '이끌림', '인생', '직화', '진지',
#              '천년', '청년', '청춘', '최고', '큰아', '큰아들', '탐나', '토핑', '행복', '행복한', '헬로', '홍대',
#              '육회','수락','호랑','갈비','무국','닭장','한식','마라','스시','여왕','돈까','초밥','카페','피자','부대찌','이태리','플러','플러스']
# # drop_name = ['과일','오늘', '오늘은','우리', '이끌', '이끌림', '인생','진지','최고''토핑', '행복', '행복한', '한식', '감성','걸작']

In [ ]:
import kiwipiepy
kiwi = kiwipiepy.Kiwi()

In [ ]:
#가게명의 앞 두글자들 중에서 명사, 형용사, 부사 확인함

short_word2 = []

for store in store_name:
    word = store[:2]
    print(word)
    short_word2.append(word)

In [ ]:
tokens = kiwi.analyze(short_word2)

remove_words = []

for token in tokens :
  word = token[0][0][0].form
  pos = token[0][0][0].tag
  if pos in ('VA','NNG','MAG','MAJ','VV','XR') and len(word) ==2 :
    remove_words.append(word)

remove_words

In [ ]:
len(short_word2), len(remove_words)

In [ ]:
remove_words = set(remove_words)
remove_words = list(remove_words)
len(remove_words)

In [ ]:
tokens = kiwi.analyze(store_name)

for token in tokens :
  word = token[0][0][0].form
  pos = token[0][0][0].tag
  if pos in ('NNG') and len(word) ==3 :
    print(word)

# 3글자 일반명사 단어들은 가게명으로 포함해도될듯

In [ ]:
tokens = kiwi.analyze(store_name)

for token in tokens :
  word = token[0][0][0].form
  pos = token[0][0][0].tag
  if pos in ('NNG') and len(word) ==4 :
    print(word)

# 4글자 일반명사 단어들은 가게명으로 포함해도될듯

In [ ]:
remove_words[:10]

In [ ]:
korean_particles = ['은', '는', '이', '가', '을', '를', '의', '한', '로', '으로', '까지', '이며', '며', '와', '과']

#위 일반명사에, 조사를 합친 경우도 제외함

In [ ]:
remove_withparticles = []

for word in remove_words :
  for particle in korean_particles:
    new_word = word + particle
    remove_withparticles.append(new_word)

- 메뉴에 들어있는 명사 제외

In [ ]:
ygy_menu_list = ['양식','일식','중식','중국집','태국음식','한식','분식','도시락','카레','커피','국밥','마라탕','마라샹궈','우동','나가사키','연어','치킨','쌀국수','닭강정','김치찜','김치찌개',
                '족발','보쌈','육회','갈비','피자','찜닭','파스타','냉면','곱창','닭발','막창','반찬','곱도리탕','순대','떡볶이','부대찌개'
                '삼겹살','분식','라면','버거','덮밥','초밥','통닭','짜장','짬뽕','탕수육','카페','와플','누들','김치','온면','디저트','라떼','고기','덮밥',
                '새우','돼지','닭','찌개','밥','타코야키','탕',
                 '양파','숙주','고수'] # 재료명 제거

In [ ]:
menu_for_store = []

for name1 in tqdm(ygy_menu_list, desc="Processing"):
        for i in range(len(name1), 1, -1):
            part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분. 글자 긴경우 부터 적용됨
            menu_for_store.append(part)

In [ ]:
menu_for_store = set(menu_for_store)
menu_for_store = list(menu_for_store)

- 최종 함수

In [ ]:
def replace_store_names(text):
    for name1 in store_name:
        # 먼저 가게 이름 전체가 텍스트에 포함되어 있는지 확인
        if name1 in text:
            # 가게 이름 전체가 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<store>", text)
        else:
            # 가게 이름의 일부가 텍스트에 포함되어 있는 경우를 확인
            for i in range(len(name1), 1, -1):
                part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
                if part in text:
                    # 가게 이름의 일부가 텍스트에 포함되어 있고, 해당 부분이 remove_words에 없으며,
                    # 해당 부분 뒤에 조사가 붙지 않은 경우에만 대체
                    if part not in (remove_words + remove_withparticles + menu_for_store) :
                        pattern = re.compile(re.escape(part), re.IGNORECASE)
                        text = pattern.sub("<store>", text)
                      # 가게 이름 일부를 대체했으면 루프 종료
    return text


In [ ]:
replace_store_names('행복한 라화방마라탕에서 맛있게 식사했어요') #행복 부분이 대체안됨

In [ ]:
# train['reply']의 답변들에 함수 적용하기 / 18000개 7분 정도 걸림

for i, review in tqdm(enumerate(train['reply']), desc="Processing", ncols=100):
    train['reply'][i] = replace_store_names(review)

In [ ]:
changed_rows = train[train['reply'].str.contains("<store>")]
changed_rows

In [ ]:
# train['review']의 리뷰들에 함수 적용하기 / 7분 정도 걸림

for i, review in tqdm(enumerate(train['review']), desc="Processing", ncols=100):
    train['review'][i] = replace_store_names(review)

In [ ]:
changed_rows = train[train['review'].str.contains("<store>")]

In [ ]:
train['review'] = train['review'].replace('<store><store> | <store> <store>','<store>')
train['reply'] = train['reply'].replace('<store><store> | <store> <store>','<store>')

#### 지점명 처리

In [ ]:
# 정규 표현식을 사용하여 "~호점" 패턴을 찾아서 삭제하는 함수
def remove_hojum(text):
    return re.sub(r'\w+\s*호점','', text)

# train 데이터프레임의 'reply' 열에 적용하여 "~호점"을 삭제
train['reply'] = train['reply'].apply(remove_hojum)
train['review'] = train['review'].apply(remove_hojum)


In [ ]:
restaurant_list = pd.read_csv(f"{DATA_PATH}fulldata_07_24_04_P_일반음식점.csv",encoding='cp949')

In [ ]:
restaurant_list['소재지전체주소']

In [ ]:
address_parts=[] #소재지 데이터에서 서울, 부산 동, 구, 리 등을

for address in restaurant_list['소재지전체주소'].astype(str):
    if address.startswith('서울'or '부산' or '청주'):
        parts = address.split(' ')
        for _ in range(0, min(4, len(parts))):  # 최대 5개 또는 주소 부분 개수만큼 반복
          part = parts[_]
          address_parts.append(part)

address_parts = set(address_parts)
address_parts = list(address_parts)

address_parts = [name for name in address_parts if all(char.isalpha() for char in name)] #알파벳으로만 (한글) 구성된 경우남김. 숫자, 특수문자 지움
address_parts.remove('')

In [ ]:
# 가게명에서 지점명 추출하기

store_etc_name = []

for store in ygy_store_lst:
    pat = re.split(r'[- &]', store)
    for name in pat[1:]:
        if name != '':
            store_etc_name.append(name)

ygy_region_name = []

for region in store_etc_name:
  if region[-1] =='점':
    ygy_region_name.append(region)

In [ ]:
address_parts2 = [] # 추출된 지역명을 지점명으로 변경

for address in address_parts :
  if len(address) < 3  :
    address = address + '점'
    address_parts2.append(address)
  else :
    address = address[:-1] + '점'
    address_parts2.append(address)

In [ ]:
len(address_parts2)

In [ ]:
region_name = ygy_region_name + address_parts2

In [ ]:
region_name = set(region_name)
region_name = list(region_name)

In [ ]:
region_name[:10]

In [ ]:
name1 = '성북미아점'

for i in range(len(name1), 1, -1):
    part = name1[:i]
    print(part)

In [ ]:
def replace_region_names(text):
    for name1 in region_name:
        if name1 in text:
            # 지점 이름이 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<region>", text)
        else :
          for i in range(len(name1), 1, -1):
              part = name1[:i]  # 지점 이름에서 처음부터 i 글자까지의 부분
              if part in text:
                  # 지점 이름의 일부가 텍스트에 포함되어 있다면 대체
                  pattern = re.compile(re.escape(part), re.IGNORECASE)
                  text = pattern.sub("<region>", text)
    return text


In [ ]:
replace_region_names('오랜만에 헬리오시티점에서 맛있게 식사했어요')

In [ ]:
replace_region_names('성북점이 강북점보다 더 맛있어요')

In [ ]:
replace_region_names('성북구가 강북구보다 더 맛있어요')

In [ ]:
for i, review in tqdm(enumerate(train['reply']), desc="Processing", ncols=100):
    train['reply'][i] = replace_region_names(review)

In [ ]:
changed_rows = train[train['reply'].str.contains("<region>")]
changed_rows

In [ ]:
for i, review in tqdm(enumerate(train['review']), desc="Processing", ncols=100):
    train['review'][i] = replace_region_names(review)

In [ ]:
changed_rows = train[train['review'].str.contains("<region>")]
changed_rows

#### 메뉴명 처리

In [ ]:
food_list = pd.read_csv(f'{DATA_PATH}전국통합식품영양성분정보(음식)표준데이터.csv', encoding='cp949')

In [ ]:
food_list = set(food_list['대표식품명'].tolist())

In [ ]:
food_list = list(food_list)

In [ ]:
ygy_menu_list = ['양식','일식','중식','중국집','태국음식','한식','분식','도시락','카레','커피','국밥','마라탕','마라샹궈','우동','나가사키','연어','치킨','쌀국수','닭강정','김치찜','김치찌개',
                '족발','보쌈','육회','갈비','피자','찜닭','파스타','냉면','곱창','닭발','막창','반찬','곱도리탕','순대','떡볶이','부대찌개'
                '삼겹살','분식','라면','버거','덮밥','초밥','통닭','짜장','짬뽕','탕수육','카페','와플','누들','김치','온면','디저트','라떼','고기','덮밥',
             '새우','돼지','닭','찌개','밥','타코야키','탕','밥','국','항정살',
                 '양파','숙주','고수'] # 재료명 제거

In [ ]:
menu_name = food_list + ygy_menu_list

In [ ]:
len(menu_name)

In [ ]:
new_food_list = []  # 새로운 음식을 담을 리스트

for food in food_list:
    if '(' in food:
        food1, food2 = food.split('(')
        new_food_list.append(food1.strip())  # 공백 제거 후 리스트에 추가
        new_food_list.append(food2.rstrip(')'))
    else:
        new_food_list.append(food)

food_list = new_food_list

In [ ]:
menu_name = set(menu_name)
menu_name = list(menu_name)

menu_name = [name for name in menu_name if all(char.isalpha() for char in name)] #알파벳으로만 (한글) 구성된 경우남김. 숫자, 특수문자 지움


- 1차 : 풀로 이름 들어간 것 대체

In [ ]:
def replace_menu_names1(text):
    # 메뉴 이름을 긴 것부터 짧은 것 순으로 정렬
    sorted_menu_names = sorted(menu_name, key=len, reverse=True)

    for name1 in sorted_menu_names:
        if name1 in text:
            # 메뉴 이름이 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<menu>", text)
    return text


In [ ]:
replace_menu_names1('부대찌개 정말 맛있었어요')

In [ ]:
replace_menu_names1('탕, 국, 마라탕, 밥 정말 맛있었어요')

- 2차 : 부분 이름 들어간 것 대체

In [ ]:
name1='허니브레드'

for i in range(len(name1),1,-1):
    part = name1[:i]
    print(part)

In [ ]:
def replace_menu_names2(text):

    for name1 in menu_name:
        for i in range(len(name1), 1, -1):
            part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
            if part in text:
                # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
                pattern = re.compile(re.escape(part), re.IGNORECASE)
                text = pattern.sub("<menu>", text)

    return text


In [ ]:
replace_menu_names2('부대 정말 맛있었어요')

In [ ]:
replace_menu_names2('떡볶 정말 맛있었어요')

- 3차 : 일반명사 제외 안해도됨. 재료명 모두 일반명사

- 최종 함수

In [ ]:
def replace_menu_names(text):
    # 메뉴 이름을 긴 것부터 짧은 것 순으로 정렬
    sorted_menu_names = sorted(menu_name, key=len, reverse=True)

    for name1 in sorted_menu_names:
        if name1 in text:
            # 메뉴 이름이 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<menu>", text)
        else:
            for i in range(len(name1), 1, -1):
                part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
                if part in text:
                    # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
                    pattern = re.compile(re.escape(part), re.IGNORECASE)
                    text = pattern.sub("<menu>", text)
                     # 대체를 하고 나서는 루프를 종료합니다.
    return text


In [ ]:
# def replace_menu_names(text):
#     # 메뉴 이름을 긴 것부터 짧은 것 순으로 정렬
#     if text is not None:
#         sorted_menu_names = sorted(menu_name, key=len, reverse=True)

#         for name1 in sorted_menu_names:
#             if name1 in text:
#                 # 메뉴 이름이 텍스트에 포함되어 있다면 대체
#                 pattern = re.compile(re.escape(name1), re.IGNORECASE)
#                 text = pattern.sub("<menu>", text)
#             else:
#                 for name1 in menu_name:
#                     for i in range(len(name1), 1, -1):
#                         part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
#                         if part in text:
#                             # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
#                             pattern = re.compile(re.escape(part), re.IGNORECASE)
#                             text = pattern.sub("<menu>", text)
#                         break

In [ ]:
for i, review in tqdm(enumerate(train['reply']), desc="Processing", ncols=100):
    train['reply'][i] = replace_menu_names(review)

In [ ]:
changed_rows = train[train['reply'].str.contains("<menu>")]
changed_rows

In [ ]:
for menu in menu_name:
    for i, review in enumerate(train['review']):
        if menu in review:
            train['review'][i] = replace_menu_names(review)

In [ ]:
changed_rows = train[train['review'].str.contains("<menu>")]
changed_rows

In [ ]:
train.replace(['<menu><menu>','<menu> <menu>'], '<menu>')

#### 기타표현 처리

- 영어, 오타, 띄어쓰기 청.춘.식.당 / 100g / 이벤트 / 24시간 / 장소 / 영업시간 등

In [ ]:
train['reply'] = train['reply'].str.replace(r'[0-9ㄱ-ㅎ]|이벤트|24시간|영업시간', '', regex=True)
train['review'] = train['review'].str.replace(r'[0-9ㄱ-ㅎ]|이벤트|24시간|영업시간', '', regex=True)


In [ ]:
train['reply']

## 증강

In [ ]:
from sklearn.utils import resample

minority_class = train[train['sentiment'] == '부정']


In [ ]:
mask_neg = train['sentiment'] == '부정'
train[mask_neg].shape[0]

In [ ]:
mask_pos = train['sentiment'] == '긍정'
train[mask_pos].shape[0]

In [ ]:
google_food = pd.read_csv(f"{DATA_PATH}playstore_food_reply_0904.csv")

In [ ]:
google_food

In [ ]:
desired_samples = train[mask_pos].shape[0] - train[mask_neg].shape[0] #pos, neg개수 동일하게 하기 위해서
oversampled_data = resample(minority_class, replace=True, n_samples=desired_samples, random_state=42)


In [ ]:
aug_train = pd.concat([train,oversampled_data],axis=0)
aug_train.shape

## 모델 학습

In [ ]:
train_ft = aug_train

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "skt/kogpt2-base-v2"
# model_name = "EasthShin/Youth_Chatbot_Kogpt2-base"

model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

tokenizer.eos_token_id # eos 토큰의 idx 반환

In [ ]:
tokenizer.tokenize(train['review'][0])

- 데이터셋 만들기

In [ ]:
train_ft = train_ft[['review','reply']]

In [ ]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.review = df["review"].tolist()
        self.reply = df["reply"].tolist()

    def __len__(self):
        return len(self.review)

    def __getitem__(self,idx):
        return "<w>" + "</s><q>" + self.review[idx] + "</s><a>" + self.reply[idx] + "</s>"

In [ ]:
def collate_fn(batch):
    x = tokenizer(batch, return_tensors="pt", padding=True)
    return {"x":x}

In [ ]:
train_dt = ChatDataset(train_ft)

In [ ]:
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=2,collate_fn=collate_fn)

In [ ]:
batch = next(iter(train_dl))
batch

In [ ]:
tokenizer.pad_token_id # 패딩토큰인 3번 빼고 계산해야겠군

- 학습 loop함수화

In [ ]:
def train_loop(dataloader,model,optimizer,loss_fn,device):
    epoch_loss = 0
    model.train()

    for batch in tqdm(dataloader):
        x = batch["x"].to(device)
        pred = model(**x).logits # batch, seq, 단어별 실수값
        n_class = pred.shape[-1] # 클래스 개수
        pred = pred[:,:-1] # eos 토큰 제외
        pred = pred.reshape(-1,n_class) # 2차원 형태로 변환

        tgt = x["input_ids"][:,1:]
        tgt = tgt.flatten() # 1차원 벡터 형태로 변환

        mask = tgt != 3 # 패딩토큰인 3번 빼고 계산해야겠군
        tgt = tgt[mask]
        pred = pred[mask]
        loss = loss_fn(pred,tgt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    return epoch_loss

- 학습하기

In [ ]:
batch_size = 2
epochs = 10
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
train_ft['review'] = train_ft['review'].astype(str)
train_ft['reply'] = train_ft['reply'].astype(str)
# train_ft['store'] = train_ft['store'].astype(str)

- 총 5750개의 데이터 학습 (증강)

In [ ]:
is_holdout = False
reset_seeds(SEED)

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=3e-5)

train_dt = ChatDataset(train_ft)
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

for i in range(epochs):
    train_loss = train_loop(train_dl, model, optimizer, loss_fn, device)
    print(train_loss)

    if is_holdout:
        break


# 0.18275894721305888

- 모델 저장하기

In [ ]:
model.save_pretrained(f"{DATA_PATH}koGPT2_keybert_masktoken_0905_2") #0

## 답변 생성

- 맞춤법 처리

In [ ]:
!git clone https://github.com/ssut/py-hanspell.git

In [ ]:
!cd py-hanspell

In [ ]:
!pip install py-hanspell/

In [ ]:
from hanspell import spell_checker

# def correct_spell(text):
#     if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
#         corrected_text = spell_checker.check(text).checked
#         return corrected_text
#     else:
#         return text

- 모델 불러오기

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# model_name = 'MLOpsEngineer/review-rate-prediction'

# model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}ygy_cont3000_youth_ver1")
model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}koGPT2_keybert_masktoken_0905_2")

In [ ]:
model_name = "skt/kogpt2-base-v2"

tokenizer = AutoTokenizer.from_pretrained(model_name, #해당 모델의 토크나이저
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024,
                                          padding_side='left')

In [ ]:
def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

In [ ]:
# @torch.no_grad()
# def chatbot(model,tokenizer,max_len,device):
#     model.eval()
#     while True:
#         text = input("소비자 > ").strip()
#         if text == "quit":
#             break
#         text = "<q>" + text + "</s><a>"
#         x = tokenizer.encode(text,return_tensors="pt").to(device) # batch, seq

#         result_ids = model.generate(x,
#                             max_length=max_len,
#                             repetition_penalty=2.0,
#                             use_cache=True,
#                             do_sample=True, # 확률적 샘플링 여부(False = greedy 방식)
#                             temperature=0.2, # 소프트맥스 온도(확률적 샘플링일 때만 작동)
#                             top_k=5) # 상위 확률 k개 기준으로 확률적 샘플링 하겠다(확률적 샘플링일 때만 작동)
#         q_len = len(text) + 1
#         text = tokenizer.decode(result_ids[0])

#         from hanspell import spell_checker

#         text = correct_spell(text)

#         # if len(text) > max_len:
#         #     text = text[:max_len]

#         print("사장님 > ",text[q_len:-4])

In [ ]:

# chatbot 함수 정의
def chatbot(model, tokenizer, max_len, review_list, store, menu, region):
    responses = []  # 결과를 저장할 리스트 초기화

    for review in tqdm(review_list):

        review = review.replace('<store>',store)
        review = review.replace('<menu>',menu)
        review = review.replace('<region>',region)


        # 대화 시작 및 종료 토큰 추가
        text = "<q>" + review + "</s><a>"

        # 입력 텍스트를 인코딩
        x = tokenizer.encode(text, return_tensors="pt")
        # max_len = 10000, max_length=max_len

        # 모델로부터 응답 생성
        result_ids = model.generate(x,
                                    max_length=max_len,
                                    num_return_sequences=1,
                                    repetition_penalty=0.9, # 1보다 낮으면 반복 허용, 1보다 클수록 창의적
                                    use_cache=True,
                                    do_sample=True, # 확률적 샘플링 여부(False = greedy 방식)
                                    temperature=1, # softmax. 1보다 낮으면 결정적, 1보다 클수록 창의적
                                    top_k=5) # 상위 확률 k개 기준으로 확률적 샘플링 하겠다(확률적 샘플링일 때만 작동)


        # 응답을 텍스트로 디코딩하여 저장
        response = tokenizer.decode(result_ids[0], skip_special_tokens=True)

        response = response.replace('<store>',store)
        response = response.replace('<menu>',menu)
        response = response.replace('<region>',region)

        response = response.split("<a>", 1)[1]
        response = correct_spell(response)
        responses.append(response)

    # 결과를 데이터프레임으로 변환
    result_df = pd.DataFrame({"Review": review_list, "Response": responses})
    return result_df


In [ ]:
review_list = [

    '맛이 있어요',
    '최고에요',
    '사장님이 친절하시네요',
    '맛은 있는데 배달이 식어서 왔어요ㅠㅠ',
    '맛이 없어요',
    '이 머리카락 들어있는거 뭔가요?',
    '배달이 너무 불친절해요',
    '최악이에요'

]

# chatbot 실행 및 결과 출력
result_df = chatbot(model, tokenizer, 512, review_list,'최가네','덮밥','서초점')

pd.DataFrame(result_df)

In [ ]:
review_list = [

    '맛이 있어요',
    '최고에요',
    '사장님이 친절하시네요',
    '맛은 있는데 배달이 식어서 왔어요ㅠㅠ',
    '맛이 없어요',
    '이 머리카락 들어있는거 뭔가요?',
    '배달이 너무 불친절해요',
    '최악이에요'

]

# chatbot 실행 및 결과 출력
result_df = chatbot(model, tokenizer, 512, review_list,'비비큐','치킨','양재점')

pd.DataFrame(result_df)

In [ ]:
train_ft.shape # 2800-900개

In [ ]:
result_df.to_csv(f'{DATA_PATH}[response]koGPT2_keybert_masktoken_0905_2a.csv',index=False)

In [ ]:
review_list = [
    # 특정 메뉴명 포함 (맛)
    '이집 피자 최애 넘나 맛있어요',
    '치킨이 바삭바삭하니 맛있네요',
    '마라샹궈 넘넘 맛있고 양도 푸짐해용! 감사합니다!',
    '간만에 술한잔생각나서 연어 주문해서 먹는데 맛있네요^^ 양이 좀적어진거같은데 다음에는 추가로 시켜야 겠어요^^ 잘먹었습니당^^',
    '짜장면 배달이 너무 느리게 와서 음식이 다 불었네요..사장님 신경좀 써주세요',
    '쌀국수 먹는데 비닐 나왔네요 반품 이런건 안하지만 청결에 신경 써주세요',
    '요기 김치찜 맛있는데 가끔 좀 짜요ㅠㅠ다음엔 육수 추가로 주심 좋겠어요',

    # 메뉴의 특성 포함 (맛)
    '닭도 쫄깃하니 맛있고 떡볶이도 맛나요👍',
    '오늘은 비계가 많은거 같아요.. 양이 상대적으로 적어서 아쉬웠어요.',
    '크림이 너무 부드럽고 맛있어요, 또 시켜먹을게요!!',
    '맛있게 먹었어요 4단계로 해도 맵지는 않아요',


    # 배달 및 기타 코멘트
    '배달 빠르게 와서 맛있게 먹었습니다!',
    '완전 만족했어요 전문 블로거인데요 양쪼끔만 더 많았으면! 종종 시켜먹을게요',
    '맛은 정말 좋았지만, 편의점가려다 배달 도착한걸 발견 못해다면 다 상한 음식 먹을뻔했네요. 노크도 없고 문자도 안주시면 음식이 온지 어떻게 알으라는건지....',
    '.................',
    '벌레 나왔어요.. 다신 안시켜먹을 것 같아요',
]

result_df = chatbot(model, tokenizer, 1024, review_list,'비비큐','치킨','서초점')

pd.DataFrame(result_df)

In [ ]:

chatbot(model,tokenizer,128,device) # quit

In [ ]:
result_df.to_csv(f'{DATA_PATH}[response]koGPT2_keybert_masktoken_0905_2b.csv',index=False)

# --------참고. 리뷰 키워드 사전학습모델

## A. Keybert로 키워드 추출

In [ ]:
!pip install keybert

### 1) skt/kobert-base-v1

In [ ]:
# from keybert import KeyBERT
# # from transformers import XLNetTokenizer, BertTokenizer, BertModel
# from tqdm import tqdm  # tqdm 추가
# from transformers import AutoModel, AutoTokenizer


# # keybert에 사용할 Kobert 모델 로드
# tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
# model = AutoModel.from_pretrained("skt/kobert-base-v1")

# # KeyBERT 모델 초기화
# keybert_model = KeyBERT(model=model)

# # 리뷰 데이터 (train['review']로 가정)
# reviews = train['review'][:50]  # train['review']에 리뷰 데이터를 넣으세요
# keywords_list = []

# # tqdm을 사용하여 진행 상황 표시
# for text in tqdm(reviews):
#     keywords = keybert_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
#     keywords_list.append(keywords)

# # # 결과 출력
# # for idx, keywords in enumerate(keywords_list):
# #     print(f"Review {idx + 1} Keywords:", keywords)

# result_df = pd.DataFrame({'Review': reviews, 'Keywords': keywords_list})
# result_df




### 2) BM-K/KoSimCSE-roberta

In [ ]:
# import pandas as pd
# from keybert import KeyBERT
# from tqdm import tqdm
# # from transformers import BertTokenizer, RobertaTokenizer, RobertaModel
# from transformers import AutoModel, AutoTokenizer



# # BM-K/KoSimCSE-roberta 모델 로드
# tokenizer = AutoTokenizer.from_pretrained("BM-K/KoSimCSE-roberta")
# model = AutoModel.from_pretrained("BM-K/KoSimCSE-roberta")

# # KeyBERT 모델 초기화
# keybert_model = KeyBERT(model=model)

# # 리뷰 데이터 (train['review']로 가정)
# reviews = train['review'][:50]  # train['review']에 리뷰 데이터를 넣으세요
# keywords_list = []

# # tqdm을 사용하여 진행 상황 표시
# for text in tqdm(reviews):
#     keywords = keybert_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=5)
#     keywords_list.append(keywords)

# # 데이터 프레임으로 변환
# result_df = pd.DataFrame({'Review': reviews, 'Keywords': keywords_list})
# result_df


### 3) distilbert-base-nli-mean-tokens
- 이 모델이 Keybert의 디폴트 모델인거 같음

In [ ]:
# from keybert import KeyBERT
# from tqdm import tqdm


# # KeyBERT 모델 초기화 (distilbert-base-nli-mean-tokens 사용)
# keybert_model = KeyBERT('distilbert-base-nli-mean-tokens')

# # 리뷰 데이터 (train['review']로 가정)
# reviews = train['review'][:50]  # train['review']에 리뷰 데이터를 넣으세요
# keywords_list = []

# # tqdm을 사용하여 진행 상황 표시
# for text in tqdm(reviews):
#     keywords = keybert_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=5)
#     keywords_list.append(keywords)

# # 결과 출력
# result_df = pd.DataFrame({'Review': reviews, 'Keywords': keywords_list})
# result_df


### 4) monologg/kobert

In [ ]:
# import pandas as pd
# from keybert import KeyBERT
# from tqdm import tqdm
# # from transformers import BertTokenizer, RobertaTokenizer, RobertaModel
# from transformers import AutoModel, AutoTokenizer



# # BM-K/KoSimCSE-roberta 모델 로드
# tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
# model = AutoModel.from_pretrained("monologg/kobert")

# # KeyBERT 모델 초기화
# keybert_model = KeyBERT(model=model)

# # 리뷰 데이터 (train['review']로 가정)
# reviews = train['review'][:50]  # train['review']에 리뷰 데이터를 넣으세요
# keywords_list = []

# # tqdm을 사용하여 진행 상황 표시
# for text in tqdm(reviews):
#     keywords = keybert_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
#     keywords_list.append(keywords)

# # 데이터 프레임으로 변환
# result_df = pd.DataFrame({'Review': reviews, 'Keywords': keywords_list})
# result_df



## B. 키위로 키워드 추출